In [22]:
import torch
import lightgbm as lgb
from tqdm import tqdm
from loaders.lending_loader import load_data, mono_list
from monotonenorm import SigmaNet, GroupSort, direct_norm
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import numpy as np

In [5]:
Xtr, Ytr, Xts, Yts = load_data(get_categorical_info=False)
monotonic_constraints = [int(i in mono_list) for i in range(Xtr.shape[1])]

In [9]:
clf = lgb.LGBMRegressor(n_estimators=10000, max_depth=5, learning_rate=.1, monotonic_constraints= monotonic_constraints)
clf.fit(Xtr, Ytr, early_stopping_rounds=200, eval_set=[(Xts, Yts)], eval_metric='mse', verbose=0)

[LightGBM] [Warning] Unknown parameter: monotonic_constraints


LGBMRegressor(max_depth=5,
              monotonic_constraints=[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              n_estimators=10000)

In [27]:
Yts.astype(int)

array([1, 0, 0, ..., 0, 0, 1])

In [28]:
acc = accuracy_score(Yts, clf.predict(Xts))
#bacc = balanced_accuracy_score(Yts, clf.predict(Xts))
acc, bacc

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [64]:

per_layer_lip = 10
width = 128

#direct_norm = lambda x, *args, **kwargs: x # make it a normal network

activation = lambda : GroupSort(2)
#activation = lambda : torch.nn.ReLU()

model = torch.nn.Sequential(
  direct_norm(torch.nn.Linear(Xtr.shape[1], width), kind="one-inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, width), kind="inf", alpha=per_layer_lip),
  activation(),
  direct_norm(torch.nn.Linear(width, 1), kind="inf", alpha=per_layer_lip),
).cuda()
#model = SigmaNet(model, sigma=per_layer_lip**4, monotone_constraints=[int(i in mono_list) for i in range(Xtr.shape[1])])

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
print('params:', sum(p.numel() for p in model.parameters()))
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.9999)


params: 68609


In [65]:
Xtrt = torch.tensor(Xtr, dtype=torch.float32).cuda()
Ytrt = torch.tensor(Ytr, dtype=torch.float32).view(-1, 1).cuda()
Xtst = torch.tensor(Xts, dtype=torch.float32).cuda()
Ytst = torch.tensor(Yts, dtype=torch.float32).view(-1, 1).cuda()

dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(Xtrt, Ytrt), batch_size=1024, shuffle=True)

bar = tqdm(range(1000))
for i in bar:
  for Xi, yi in dataloader:
    y_pred = model(Xi)
    losstr = torch.nn.functional.mse_loss(y_pred, yi)
    optimizer.zero_grad()
    losstr.backward()
    optimizer.step()
    scheduler.step()

  with torch.no_grad():
    y_predts = model(Xtst)
    lossts = torch.nn.functional.mse_loss(y_predts, Ytst)
    bar.set_description(f'Loss: {losstr.item():.4f} {lossts.item():.4f}, RMSE: {(losstr.item()**.5):.4f} {(lossts.item()**.5):.4f}')

Loss: 0.0050 0.0397, RMSE: 0.0710 0.1992:  27%|██▋       | 273/1000 [01:56<05:11,  2.33it/s]


KeyboardInterrupt: 